# ДЗ по мат. стату

**Выполнил: Копин Борис Александрович**

 **Задание**: Вам необходимо реализовать на языке R функции вычисления доверительных интервалов для параметров нормального распределения для случаев, рассмотренных на занятии.

1) Вычисление доверительного интервала для среднего:
* Дисперсия известна
* Дисперсия неизвестна

2) Вычисление доверительного интервала для дисперсии:
* Мат. ожидаение известно
* Мат. ожидаение неизвестно

3) Вычисление доверительных интервалов для разницы мат. ожиданий двух выборок. По умолчанию следует проверять равенство дисперсий при помощи подхода с использованием F-теста, а далее в зависимости от результата:
* либо с предположением о неравенстве дисперсий,
* либо с альтернативой.

Также должна присутствовать возможность задать предположение о равенстве дисперсий через параметры функции явно.

# Практическая часть

## Функция 1

In [3]:
expectation.respect.to.variance <- function(sample, variance, alpha) {
    sample.mean <- mean(sample)
    .T <- (qnorm(1 - alpha/2, mean=0, sd=1) * variance) / sqrt(length(sample))
    
    left_bound <- sample.mean - .T
    right_bound <- sample.mean + .T
    return(c(left_bound, right_bound))
}


In [4]:
expectation.respect.to.unknown.variance <- function(sample, alpha) {
    .n <- length(sample)
    .mean <- mean(sample)
    
    .S <- sqrt((1/(.n-1)) * sum((sample - .mean)**2))

    .T <- (qt(1 - alpha/2, .n-1) * .S) / sqrt(.n)
    
    left_bound <- .mean - .T
    right_bound <- .mean + .T
    return(c(left_bound, right_bound))
}

In [5]:
func1 <- function(sample, alpha, sigma) {
    if(!missing(sigma)) {
        ### Случай 1: Неизвестное мат. ожидание
        ### при известной дисперсии
        return (expectation.respect.to.variance(sample, sigma, alpha))
    } else {
        ### Случай 3: Неизвестное мат. ожидание
        ### при неизвестной дисперсии
        return (expectation.respect.to.unknown.variance(sample, alpha))
    }
}

In [6]:
{
    n <- 1000
    sigma <- 3
    mu <- 0
    alpha <- 0.05
    sample <- rnorm(n, mean=mu, sd=sigma)
    
    print(func1(sample = sample, alpha = alpha))
    print(func1(sample = sample, alpha = alpha, sigma))
}

[1] -0.1860721  0.1896151
[1] -0.184167  0.187710


## Функция 2

In [7]:
variance.respect.to.expectation <- function(sample, expectation, alpha) {
    .n <- length(sample)
    .T <- sum((sample - expectation)**2)

    left_bound <- sqrt(.T/qchisq(1 - alpha/2, .n))
    right_bound <- sqrt(.T/qchisq(alpha/2, .n))
    return(c(left_bound, right_bound))
}

In [8]:
variance.respect.to.unknown.mean <- function(sample, alpha) {
    .n <- length(sample)
    .T <- sum((sample[1:.n-1] - mean(sample[1:.n-1]))**2)

    left_bound <- sqrt(.T/qchisq(1 - alpha/2, .n-1))
    right_bound <- sqrt(.T/qchisq(alpha/2, .n-1))

    return(c(left_bound, right_bound))
}

In [9]:
func2 <- function(sample, alpha, mu) {
    if(!missing(mu)) {
        ### Случай 2: Неизвестная дисперсия
        ### при известном мат. ожидании
        return (variance.respect.to.expectation(sample, mu, alpha))
    } else {
        ### Случай 4: Неизвестная дисперсия
        ### при известном мат. ожидании
        return (variance.respect.to.unknown.mean(sample, alpha))
    }
}

In [10]:
{
    n <- 1000
    sigma <- 3
    mu <- 0
    alpha <- 0.05
    sample <- rnorm(n, mean=mu, sd=sigma)
    
    print(func2(sample = sample, alpha = alpha))
    print(func2(sample = sample, alpha = alpha, mu))
}

[1] 2.948739 3.219148
[1] 2.948029 3.218232


## Функция 3

In [11]:
fisher.test <- function(sample1, sample2) {
    .n1 <- length(sample1)
    .mean1 <- mean(sample1)
    .s1 <- (1/(.n1-1)*sum((sample1 - .mean1)**2))
    
    .n2 <- length(sample2)
    .mean2 <- mean(sample2)
    .s2 <- (1/(.n2-1)*sum((sample2 - .mean2)**2))
    
    return(df(.s1/.s2, .n1-1, .n2-1))
}

In [12]:
expectations.subs.respect.to.variances <- function(sample1, variance1, sample2, variance2, alpha) {
    .n1 <- length(sample1)
    .mean1 <- mean(sample1)
    
    .n2 <- length(sample2)
    .mean2 <- mean(sample2)
    
    .S <- sqrt(variance1**2/.n1 + variance2**2/.n2)

    .T <- qnorm(1 - alpha/2) * .S
    
    left_bound <- .mean1 - .mean2 - .T
    right_bound <- .mean1 - .mean2 + .T
    return(c(left_bound, right_bound))
}

In [13]:
expectations.subs.respect.to.same.unknown.variances <- function(sample1, sample2, alpha) {
    .n1 <- length(sample1)
    .mean1 <- mean(sample1)
    
    .n2 <- length(sample2)
    .mean2 <- mean(sample2)
    
    .S1 <- sqrt((sum((sample1 - .mean1)**2) + sum((sample2 - .mean2)**2))/(.n1 + .n2 - 2))
    .S2 <- sqrt(1/.n1 + 1/.n2)

    .T.left <- qt(1 - alpha/2, .n1 + .n2 -2) * .S1 * .S2
    .T.right <- qt(alpha/2, .n1 + .n2 -2) * .S1 * .S2
    
    left_bound <- .mean1 - .mean2 - .T.left
    right_bound <- .mean1 - .mean2 - .T.right
    return(c(left_bound, right_bound))
}

In [14]:
expectations.subs.respect.to.unknown.variances <- function(sample1, sample2, alpha) {
    .n1 <- length(sample1)
    .mean1 <- mean(sample1)
    .s1 <- (1/(.n1-1)*sum((sample1 - .mean1)**2))
    
    .n2 <- length(sample2)
    .mean2 <- mean(sample2)
    .s2 <- (1/(.n2-1)*sum((sample2 - .mean2)**2))
    
    .t <- .s1/.n1 + .s2/.n2
    .T <- sqrt(.t)
    
    .k <- (.t**2) / ( (.s1/.n1)**2/(.n1 - 1) + (.s2/.n2)**2/(.n2 - 1) )

    .T.left <- qt(1 - alpha/2, .k) * .T
    .T.right <- qt(alpha/2, .k) * .T
    
    left_bound <- .mean1 - .mean2 - .T.left
    right_bound <- .mean1 - .mean2 - .T.right
    return(c(left_bound, right_bound))
}

In [15]:
func3 <- function(sample1, sample2, sigma1, sigma2, alpha, explicit=FALSE, threshold = 0.05) {
    # при неизвестных дисперсиях
    if(!missing(sigma1) & !missing(sigma2)) {
        ### Случай 1: Неизвестная разница мат. ожиданий
        ### при известных дисперсиях
        print("Случай 1.")
        expectations.subs.respect.to.variances(sample1 = sample1,
                                               variance1 = sigma1,
                                               sample2 = sample2,
                                               variance2 = sigma2,
                                               alpha = alpha)
    } else {
        hipotesis <- explicit
        
        if (!hipotesis) {
            # проверяем равенство дисперсий Ф-тестом
            print("проверяем равенство дисперсий Ф-тестом")
            hipotesis <- (fisher.test(sample1 = sample1, sample2 = sample2) > threshold)
        }

        if (hipotesis) {
            ### Случай 2: Неизвестная разница мат. ожиданий
            ### при неизвестных, но равных дисперсиях
            print("Случай 2.")
            expectations.subs.respect.to.same.unknown.variances(sample1 = sample1,
                                                                sample2 = sample2,
                                                                alpha = alpha)
        } else {
            ### Случай 3: Неизвестная разница мат. ожиданий
            ### при неизвестных, но не равных дисперсиях
            print("Случай 3.")
            expectations.subs.respect.to.unknown.variances(sample1 = sample1,
                                                           sample2 = sample2,
                                                           alpha = alpha)
        }
    }
    
    
}

In [16]:
{
    n <- 1000
    alpha <- 0.05

    sigma1 <- 3
    sigma2 <- 5

    mu1 <- 0
    sample1 <- rnorm(n, mean=mu1, sd=sigma1)

    mu2 <- 0
    sample2 <- rnorm(n, mean=mu2, sd=sigma2)
    
    print(func3(sample1 = sample1, sample2 = sample2, alpha = alpha))
    print(func3(sample1 = sample1, sample2 = sample2,
                sigma1 = sigma1, sigma2 = sigma2, alpha = alpha))
}

[1] "проверяем равенство дисперсий Ф-тестом"
[1] "Случай 3."
[1] -0.4484157  0.2877726
[1] "Случай 1."
[1] -0.4417211  0.2810779


# Исследовательская часть

In [290]:
n <- 1000
sigma <- 3
mu <- 0
alpha <- 0.05
sample <- rnorm(n, mean=mu, sd=sigma)

## Одновыборочный случай

### Случай 1: Неизвестное мат. ожидание при известной дисперсии

Доверительный интервал для $\mu$:
$(\ \bar{X} - \frac{Z_{1-\alpha/2}*\sigma}{\sqrt{n}}\ ,\ \bar{X} + \frac{Z_{1-\alpha/2}*\sigma}{\sqrt{n}}\ )$

In [291]:
confidence.interval <- expectation.respect.to.variance(sample = sample, variance = sigma, alpha = alpha) 
c(confidence.interval[1] < mu, mu < confidence.interval[2])
confidence.interval

[1] TRUE TRUE

[1] -0.1833304  0.1885467

### Случай 2: Неизвестная дисперсия при известном мат. ожидании

Доверительный интервал для $\sigma$: $(\sqrt{\frac{\sum_{i=1}^{n}(X_i-\mu)^2}{\chi^2_{1-\alpha/2, n}}},\sqrt{\ \frac{\sum_{i=1}^{n}(X_i-\mu)^2}{\chi^2_{\alpha/2, n}}})$

In [292]:
confidence.interval <- variance.respect.to.expectation(sample = sample, expectation = mu, alpha = 0.05) 
c(confidence.interval[1] < sigma, sigma < confidence.interval[2])
confidence.interval

[1] TRUE TRUE

[1] 2.820486 3.078998

### Случай 3: Неизвестная дисперсия при неизвестном мат. ожидании

Доверительный интервал для $\sigma$, $\mu$ как выборочное среднее:
$(\sqrt{\frac{\sum_{i=1}^{n}(X_i-\bar{X})^2}{\chi^2_{1-\alpha/2, n-1}}},\sqrt{\ \frac{\sum_{i=1}^{n}(X_i-\bar{X})^2}{\chi^2_{\alpha/2, n-1}}})$

In [293]:
confidence.interval <- variance.respect.to.sample.mean(sample = sample, alpha = alpha) 
c(confidence.interval[1] < sigma, sigma < confidence.interval[2])
confidence.interval

[1] TRUE TRUE

[1] 2.821298 3.080021

### Случай 4: Неизвестное мат. ожидание при неизвестной дисперсии

Доверительный интервал для $\mu$, где $\sigma$ как выборочная дисперсия: $(\bar{X} - \frac{t_{1-\alpha/2, n-1}*S}{\sqrt{n}}, \frac{t_{1-\alpha/2, n-1}*S}{\sqrt{n}} + \bar{X} )$, где ${S}={\sqrt{\frac{1}{n-1}*\sum_{i=1}^{n}(X_i-\bar{X})^2}}$

In [294]:
confidence.interval <- expectation.respect.to.unknown.variance(sample = sample, alpha = alpha) 
c(confidence.interval[1] < mu, mu < confidence.interval[2])
confidence.interval

[1] TRUE TRUE

[1] -0.1801746  0.1853910

## Двувыборочный случай

In [295]:
mu1 <- 0
sigma1 <- 3
sample1 <- rnorm(n, mean=mu1, sd=sigma1)

mu2 <- 0
sigma2 <- 3
sample2 <- rnorm(n, mean=mu2, sd=sigma2)

### Случай 1: Неизвестная разница мат. ожиданий при известных дисперсиях

Тогда получаем доверительный интервал для $\mu_1$ - $\mu_2$  в интервале: $$(\ \bar{X}-\bar{Y}-Z_{1-\alpha/2}*\sqrt{\sigma_1^2/n_1+\sigma_2^2/n_2},\ \bar{X}-\bar{Y}+Z_{1-\alpha/2}*\sqrt{\sigma_1^2/n_1+\sigma_2^2/n_2}\ )$$

In [296]:
confidence.interval <- expectations.subs.respect.to.variances(sample1 = sample1, variance1 = sigma1,
                                                              sample2 = sample2, variance2 = sigma2,
                                                              alpha = alpha) 
c(confidence.interval[1] < mu1-mu2, mu1-mu2 < confidence.interval[2])
confidence.interval

[1] TRUE TRUE

[1] -0.3990947  0.1268188

### Случай 2: Неизвестная разница мат. ожиданий при неизвестных, но равных дисперсиях

Тогда получаем доверительный интервал для $\mu_1$ - $\mu_2$  в интервале: $$(\bar{X} - \bar{Y} - t_{1-\alpha/2, n_1 + n_2 - 2}*S*\sqrt{1/n_1 + 1/n_2}, \bar{X} - \bar{Y} - t_{\alpha/2, n_1 + n_2 - 2}*S*\sqrt{1/n_1 + 1/n_2})$$, при
$$S=\sqrt{\frac{\sum_{i=1}^{n1}(X_i-\bar{X})^2 + \sum_{i=1}^{n2}(Y_i-\bar{Y})^2}{n1+n2-2}}$$

In [297]:
sigma <- 3

mu1 <- 0
sample1 <- rnorm(n, mean=mu1, sd=sigma)

mu2 <- 0
sample2 <- rnorm(n, mean=mu2, sd=sigma)

In [298]:
confidence.interval <- expectations.subs.respect.to.same.variances(sample1 = sample1, sample2 = sample2, alpha = alpha) 
c(confidence.interval[1] < mu1-mu2, mu1-mu2 < confidence.interval[2])
confidence.interval

[1] TRUE TRUE

[1] -0.2972389  0.2193956

### Случай 3: Неизвестная разница мат. ожиданий при неизвестных, но не равных дисперсиях

Тогда получаем доверительный интервал для $\mu_1$ - $\mu_2$  в интервале:
$$(\bar{X} - \bar{Y} - t_{1-\alpha/2, k}*T, \bar{X} - \bar{Y} - t_{\alpha/2, k}*T)$$
$$T = \sqrt{s_1^2/n_1+s_2^2/n_2}$$
$$k=\frac{(s_1^2/n_1+s_2^2/n_2)^2}{(s_1^2/n_1)^2/(n_1-1)+(s_2^2/n_2)^2/(n_2-1)}$$
$$s_1^2=\frac{1}{n_1-1}*\sum_{i=1}^{n_1}(X_i-\bar{X})^2$$
$$s_2^2=\frac{1}{n_2-1}*\sum_{i=1}^{n_2}(Y_i-\bar{Y})^2$$

In [299]:
sigma1 <- 3
sigma2 <- 5

mu1 <- 0
sample1 <- rnorm(n, mean=mu1, sd=sigma1)

mu2 <- 0
sample2 <- rnorm(n, mean=mu2, sd=sigma2)

In [300]:
confidence.interval <- expectations.subs.respect.to.unknown.variances(sample1 = sample1, sample2 = sample2, alpha = alpha) 
c(confidence.interval[1] < mu1-mu2, mu1-mu2 < confidence.interval[2])
confidence.interval

[1] TRUE TRUE

[1] -0.1344695  0.5793836

### Проверка гипотез про равенство дисперсий

Сначала считаем выборочные дисперсии:
$$s_1^2=\frac{1}{n_1-1}*\sum_{i=1}^{n_1}(X_i-\bar{X})^2$$
$$s_2^2=\frac{1}{n_2-1}*\sum_{i=1}^{n_2}(Y_i-\bar{Y})^2$$

Далее строим статистику: $$F=s_1^2/s_2^2 \sim F(n-1, m-1)$$

In [301]:
sigma1 <- 3
sigma2 <- 3

mu1 <- 0
sample1 <- rnorm(n, mean=mu1, sd=sigma1)

mu2 <- 0
sample2 <- rnorm(n, mean=mu2, sd=sigma2)

In [302]:
fisher.test(sample1 = sample1, sample2 = sample2) > 0.05

[1] TRUE

In [303]:
sigma1 <- 4
sigma2 <- 4

mu1 <- 0
sample1 <- rnorm(n, mean=mu1, sd=sigma1)

mu2 <- 0
sample2 <- rnorm(n, mean=mu2, sd=sigma2)

fisher.test(sample1 = sample1, sample2 = sample2) > 0.05

[1] TRUE

## Refferences:
* центральная статистика - https://en.wikipedia.org/wiki/Pivotal_quantity
* нормальное распределение - https://ru.wikipedia.org/wiki/%D0%9D%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5
* Хи-квадрат https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%85%D0%B8-%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82
* Chi-Squared Distribution - https://stat.ethz.ch/R-manual/R-devel/library/stats/html/Chisquare.html
* The Student t Distribution - https://stat.ethz.ch/R-manual/R-devel/library/stats/html/TDist.html
* Распределение фишера - https://stat.ethz.ch/R-manual/R-devel/library/stats/html/Fdist.html
* The Normal Distribution - https://stat.ethz.ch/R-manual/R-devel/library/stats/html/Normal.html